In [1]:
# подключение пакетов
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import json
import datetime
import re
import glob

In [ ]:
# процедура для получения ссылки на страницу сайта с объявление о продаже автомобиля
# пороцедура получает ссылку на страницу с перечнем объявлений, получает ссылки на
# на них и возвращает в виде списка

def get_links(url, number):
    temp = url+'&page='+str(number)
    response = requests.get(temp, headers={'User-Agent': 'Mozilla/5.0'})
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', class_='Link ListingItemTitle__link')
    return [link['href'] for link in links]

In [2]:
# процедура для получения данных со страницы
# получает ссылку и список с перечнем полей, со страницы сайта получает
# данные и возвращает в виде словаря

def get_page(link, seq):
    CarInfo = dict.fromkeys(seq)
    try:
        response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            json_temp_is = json.loads(
                soup.find('script', id='initial-state').string)
            json_temp = json.loads(
                soup.find('script', type='application/ld+json').string)
        except:
            json_temp = ''

        CarInfo['car_url'] = link
        try:
            CarInfo['price'] = json_temp['offers']['price']
        except:
            CarInfo['price'] = ''
        try:
            CarInfo['priceCurrency'] = json_temp['offers']['priceCurrency']
        except:
            CarInfo['priceCurrency'] = ''
        try:
            CarInfo['numberOfDoors'] = json_temp['numberOfDoors']
        except:
            CarInfo['numberOfDoors'] = ''
        try:
            CarInfo['complectation_dict'] = str(
                json_temp_is['card']['vehicle_info']['complectation'])
        except:
            CarInfo['complectation_dict'] = ''
        try:
            CarInfo['equipment_dict'] = str(
                json_temp_is['card']['vehicle_info']['equipment'])
        except:
            CarInfo['equipment_dict'] = ''
        try:
            CarInfo['brand'] = json_temp['brand']
        except:
            CarInfo['brand'] = ''
        try:
            CarInfo['model_name'] = json_temp_is['card']['vehicle_info']['model_info']['name']
        except:
            CarInfo['model_name'] = ''
        try:
            CarInfo['name'] = json_temp_is['card']['vehicle_info']['tech_param']['human_name']
        except:
            CarInfo['name'] = ''
        try:
            CarInfo['modelDate'] = json_temp['modelDate']
        except:
            CarInfo['modelDate'] = ''
        try:
            CarInfo['model_info'] = str(
                json_temp_is['card']['vehicle_info']['model_info'])
        except:
            CarInfo['model_info'] = ''
        try:
            CarInfo['enginePower'] = json_temp['vehicleEngine']['enginePower']
        except:
            CarInfo['enginePower'] = ''
        try:
            CarInfo['vendor'] = json_temp_is['card']['vehicle_info']['vendor']
        except:
            CarInfo['vendor'] = ''
        try:
            CarInfo['description'] = json_temp['description']
        except:
            CarInfo['description'] = ''
        try:
            CarInfo['bodyType'] = json_temp['bodyType']
        except:
            CarInfo['bodyType'] = ''
        try:
            CarInfo['super_gen'] = str(
                json_temp_is['card']['vehicle_info']['tech_param'])
        except:
            CarInfo['super_gen'] = ''
        try:
            CarInfo['vehicleConfiguration'] = json_temp['vehicleConfiguration']
        except:
            CarInfo['vehicleConfiguration'] = ''
        try:
            CarInfo['color'] = json_temp['color']
        except:
            CarInfo['color'] = ''
        try:
            CarInfo['fuelType'] = json_temp['fuelType']
        except:
            CarInfo['fuelType'] = ''
        try:
            CarInfo['image'] = json_temp['image']
        except:
            CarInfo['image'] = ''
        try:
            CarInfo['productionDate'] = json_temp['productionDate']
        except:
            CarInfo['productionDate'] = ''
        try:
            CarInfo['vehicleTransmission'] = json_temp['vehicleTransmission']
        except:
            CarInfo['vehicleTransmission'] = ''
        try:
            CarInfo['engineDisplacement'] = json_temp['vehicleEngine']['engineDisplacement']
        except:
            CarInfo['engineDisplacement'] = ''
        try:
            CarInfo['parsing_unixtime'] = round(time.time())
        except:
            CarInfo['parsing_unixtime'] = ''
        try:
            CarInfo['sell_id'] = soup.find(
                'div', class_='CardHead__infoItem CardHead__id').text.replace(u'№ ', u'')
        except:
            CarInfo['sell_id'] = ''
        try:
            CarInfo['views'] = soup.find(
                'div', class_='CardHead__infoItem CardHead__views').text.split(' ')[0]
        except:
            CarInfo['views'] = ''
        try:
            CarInfo['date_publication'] = soup.find_all(
                'div', class_='CardHead__infoItem CardHead__creationDate')[0]
        except:
            CarInfo['date_publication'] = ''

        try:
            all_blocks = soup.find_all('ul', class_='CardInfo')[0]
            bloks = all_blocks.find_all('span', class_='CardInfoRow__cell')
            car_info_temp = [link.text for link in bloks[:len(bloks)-2]]
        except:
            car_info_temp = ''

        panel = {}
        j = 0
        for item in car_info_temp:
            if j % 2 == 0:
                temp = item
                panel[temp] = ''
            if j % 2 != 0:
                panel[temp] = item
            j += 1

        try:
            CarInfo['mileage'] = panel['Пробег'].replace(
                u'\xa0', u'').replace(u'км', u'')
        except:
            CarInfo['mileage'] = ''
        try:
            CarInfo['Привод'] = panel['Привод']
        except:
            CarInfo['Привод'] = ''
        try:
            CarInfo['Руль'] = panel['Руль']
        except:
            CarInfo['Руль'] = ''
        try:
            CarInfo['Состояние'] = panel['Состояние']
        except:
            CarInfo['Состояние'] = ''
        try:
            CarInfo['Владельцы'] = panel['Владельцы'].replace(u'\xa0', u' ')
        except:
            CarInfo['Владельцы'] = ''
        try:
            CarInfo['ПТС'] = panel['ПТС']
        except:
            CarInfo['ПТС'] = ''
        try:
            CarInfo['Владение'] = panel['Владение']
        except:
            CarInfo['Владение'] = ''
        try:
            CarInfo['Таможня'] = panel['Таможня']
        except:
            CarInfo['Таможня'] = ''

    except:
        raise
    return CarInfo

In [17]:
# процедура которая возвращает со страницы сайта количество закладок с объявлениями
# получает ссылку на страницу по "рамочным" условиям находит и возвращает
# количество закладок со страницами с объявлениями

def get_numbe(url):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        return soup.find('span', class_='ButtonWithLoader__content').text
    except:
        return ' No '

In [ ]:
# Получаем ссылки с объявлениями по продаже автомобилей, для чего:
# формируем набор ссылок на страницы со списками объявлений, таким образом,
# что бы в одном наборе не оказалось более 99 страниц (ограничение сайта)
# ссылки формируются методом комбинации параметров фильтров и количества
# закладок со страницами с объявлениями

# задаём начальные условия для фильтров в словарях
link_list = []
root = ['https://auto.ru/cars/vendor-european/used/?',
        'https://auto.ru/cars/vendor-japanese/used/?']
transmission = ['MECHANICAL', 'ROBOT', 'AUTOMATIC', 'VARIATOR', 'AUTO']
engine_group = ['GASOLINE', 'DIESEL', 'HYBRID']
gear_type = ['FORWARD_CONTROL', 'REAR_DRIVE', 'ALL_WHEEL_DRIVE']

# отмета времени начала работы
print("Старт:", time.ctime(time.time()))

# комбинируем условия и получаем ссылки на страницы с объявлениями
# диапазон лет для поиска задается отдельно

for year in range(2000, 2012):
    for item_root in root:
        for item_gear_type in gear_type:
            for item_transmission in transmission:
                for item_engine_group in engine_group:
                    url = item_root+'year_from='+str(year)+'&year_to='+str(
                        year+1)+'&gear_type='+item_gear_type+'&transmission='+item_transmission+'&engine_group='+item_engine_group
                    tempn = re.findall('\s(.*?)\s', get_numbe(url))[0]
                    if tempn != 'No':
                        if int(tempn) % 37 == 0:
                            i = int(tempn)//37
                        else:
                            i = int(tempn)//37+1
                        for k in range(i):
                            link_list = link_list + get_links(url, k)
# отметка времени завершения                            
print("Финиш:", time.ctime(time.time()))

In [27]:
# сохраняем набор ссылок в файл, на случай непредвиденной ситуации
# что бы было возможно возобновить парсинг не получая набор вновь

File = open ('link_list_2000_1012_year_0.txt', 'w')
for element in link_list:
    File.write(element)
    File.write('\n')
File.close()

In [3]:
# чтение списка ссылок из файла

link_list = [line.rstrip('\n') for line in open('link_list_2000_1012_year_0.txt')]
len(link_list)

37353

In [ ]:
# получаем и сохраняем в датасет инормацию из объявлений

# начальные параметры
temp_dict = {}
num_index = 0
seq = ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen',
       'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владение', 'Владельцы', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня', 'price', 'views', 'date_publication']
df = pd.DataFrame(columns=seq)
df = df.iloc[0:0]

# отмета времени начала работы
print("Старт:", time.ctime(time.time()))

for j in range(len(link_list)):
    df.loc[num_index] = get_page(link_list[j], seq)
    num_index += 1
    time.sleep(1.1)

# отметка времени завершения 
print("Финиш:", time.ctime(time.time()))

In [ ]:
# сохранение датасета в файл

fn = 'Parsed_2000_2012_new'+str(0)+'.csv'
df.to_csv(fn,sep = ';')

In [ ]:
# объединение файлов за разные периоды времени в один

# чтаем списко файлов из директории
files = glob.glob('*.csv')
files
combined = pd.DataFrame()

# читаем файлы в датасет
for file in files:
    data = pd.read_csv(file, sep = ";")
    combined = pd.concat([combined, data])

# удаляем дубликаты и записываем результат в итоговый файл
combined = combined.reset_index(drop=True)
combined = combined.drop('Unnamed: 0', axis = 1)
clined = combined.drop_duplicates(subset=['bodyType', 'brand', 'car_url'])
clined.reset_index(inplace=True, drop=True) 
clined.to_csv( "combined_csv.csv", index=False )